# HW2: Text Generative Models

In this assignment we will see some generative models for text: CharRNN, Transformers and Chatbots. Training text models is very time consuming, and uses a ton of data. The really good models also tend to be very large, so we will stick to pretrained models. Those can still be excellent to generate totally new text!

## Word Embeddings

Embeddings are numeric representations for non-numeric data. In our case we look for embeddings for words. A simple kind of embedding is One-Hot Encoding, where we put a `1` in a vector of all `0`s at the index of the word in the vocabulary.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1" width="50%"/>

But that can be very wasteful and also doesn't encode any relationship between the words.

To learn semantic relationship a few unsupervised algorithms were proposed. In class we've discussed Continuous Bag of Words and Skip-Gram. Essentially these will mask out part of a sentence and ask the model to predict the missing part. This way the model learns about the context words are used in sentences as well as relationships.

Embedding for a word is a vector of numbers:

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" width="50%" />

Luckily many of the world leaders in natural language processing have pretrained word embeddings learned on huge corpora, so we don't have to do it ourselves.

Allison Parrish of NYU showed some very interesting uses for word embeddings for poetry generation: https://www.youtube.com/watch?v=L3D0JEA1Jdc breeze through this StrangeLoop talk for inspiration. I encourage you to try these methods towards you own generative work.

`chakin` is a helper tool for downloading pretrained embeddings:

In [0]:
!pip install -q chakin progressbar2 textgenrnn

In [0]:
import chakin
import progressbar
import numpy as np

These are the available models:

In [5]:
chakin.search('English')

                   Name  Dimension  ... Language    Author
2          fastText(en)        300  ...  English  Facebook
11         GloVe.6B.50d         50  ...  English  Stanford
12        GloVe.6B.100d        100  ...  English  Stanford
13        GloVe.6B.200d        200  ...  English  Stanford
14        GloVe.6B.300d        300  ...  English  Stanford
15       GloVe.42B.300d        300  ...  English  Stanford
16      GloVe.840B.300d        300  ...  English  Stanford
17    GloVe.Twitter.25d         25  ...  English  Stanford
18    GloVe.Twitter.50d         50  ...  English  Stanford
19   GloVe.Twitter.100d        100  ...  English  Stanford
20   GloVe.Twitter.200d        200  ...  English  Stanford
21  word2vec.GoogleNews        300  ...  English    Google

[12 rows x 7 columns]


Let's download GLoVE embeddings:

In [6]:
chakin.download(number=11)

Test: 100% ||                                      | Time:  0:06:26   2.1 MiB/s


'./glove.6B.zip'

We only need one file (the smallest dimension one):

In [8]:
!unzip glove.6B.zip glove.6B.50d.txt

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

These files contain the embedding values for each word in the vocabulary:

In [9]:
!head -5 glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581
, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392
. 0.15164 0.30177 -0.16763 0.17684 0.31719 0.33973 -0.43478 -0.31086 -0.44999 -0.29486 0.16608 0.11963 -0.41328 -0.42353

Let's load them into memory and organize a bit:

In [0]:
w2vec_lines = open('glove.6B.50d.txt','rt', encoding='utf-8').read().split('\n')

In [11]:
w2v_emb_dict = dict()
pbar = progressbar.ProgressBar(max_value=100000)
for i,l in enumerate(w2vec_lines[1:100000]):
    w,emb = l.split(' ', 1)
    w2v_emb_dict[w] = np.fromstring(emb, sep=' ')
    pbar.update(i+1)
pbar.finish()

100% (100000 of 100000) |################| Elapsed Time: 0:00:01 Time:  0:00:01


These would be the first most commonly used tokens in the vocabulary:

In [12]:
list(w2v_emb_dict.keys())[:10]

[',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s", 'for']

## Word Analogies and Similarities

Embeddings carry semantic information in their numeric encoding. Exploring this semantic space can be fun, for example looking for similarities.

Cosine similarity is measuring the angle between vectors. 

<img src="https://miro.medium.com/max/2432/1*Acs3Kbrrrb4d3fqMlGhMcQ.png"/>

Our embeddings are normalized vectors so looking at the angle between two vectors reveals how far away they are from one another in the high-dimensional embdding space:

<img src="https://www.oreilly.com/library/view/statistics-for-machine/9781788295758/assets/2b4a7a82-ad4c-4b2a-b808-e423a334de6f.png" />

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

w2v_emb_dict_keys = list(w2v_emb_dict.keys())
w2v_emb_dict_values = np.array(list(w2v_emb_dict.values()))

def find_nearest(w):
    return w2v_emb_dict_keys[cosine_similarity(w2v_emb_dict[w].reshape(1,-1), w2v_emb_dict_values)[0].argsort()[-2]]
def find_nearest_top_k(v, k=5):
    return [w2v_emb_dict_keys[w] for w in cosine_similarity(v.reshape(1,-1), w2v_emb_dict_values)[0].argsort()[-k:].tolist()[::-1]]

Let's start by looking at closest neighbors:

In [14]:
find_nearest('paris')

'france'

In [15]:
find_nearest('big')

'bigger'

In [16]:
find_nearest('hello')

'goodbye'

In [17]:
find_nearest('learning')

'teaching'

Now let's consider "**word analogies**", e.g. completing the sentence: "Paris is to France like Rome is to ___" \(Italy\)

To explain this geometrically:

<img src="https://miro.medium.com/max/2632/1*EOVxNmHkrsPQ7Q44N0OiQg.png" width="60%" />

The offset vector between "paris" and "france" is the "Captial of" vector, and when we apply it to "rome" we expect to get "italy".

In [35]:
find_nearest_top_k(w2v_emb_dict['shirt'] - w2v_emb_dict['clothing'] + w2v_emb_dict['bowl'], 5)

['bowl', 'crimson', 'afc', 'gator', 'super']

In [34]:
find_nearest_top_k(w2v_emb_dict['book'] - w2v_emb_dict['reading'] + w2v_emb_dict['tv'], 5)

['tv', 'television', 'movie', 'hbo', 'movies']

Complete the following analogies:
1. sushi-rice is like pizza-_nachos__
2. sushi-rice is like steak-_don't work__
3. shirt-clothing is like phone-_don't work__
4. shirt-clothing is like bowl-__don't work_
5. book-reading is like TV-_don't work__

Try to find analogies that don't work.



## Char RNN

CharRNN is a simple recurrent neural network architecture that works on the character level (not words). It's surprisingly powerful at generating text. These were popularized by [Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

<img src="http://karpathy.github.io/assets/rnn/charseq.jpeg" width="50%"/>

The `textgenrnn` package is a convnient way to train and generate with CharRNNs. Here we're using its built in model. They have multiple models [published](https://github.com/minimaxir/textgenrnn/tree/master/weights) trained on different corpora.

People created some very cool projects with it: https://github.com/minimaxir/textgenrnn#projects

# New Section

In [42]:
from textgenrnn import textgenrnn

textgen = textgenrnn()
textgen.train_from_file('cnn.story', num_epochs=1)
textgen.generate()

48 texts collected.
Training on 4,495 character sequences.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
35/35 [==============================] - 12s 335ms/step - loss: 1.9053
####################
Temperature: 0.2
####################






####################
Temperature: 0.5
####################
Obeidols of Land -- Asso the films costupies and "enderselfilites for the fire will been that can at movies that the films causing a the film of and can be based over the ruina form who as it get who located the films to the rape and "Zero The films are that lately the bigg of the film bot as "mov





####################
Temperature: 1.0
####################






 The ass to the films on Dark Depresier on The films and on the sony people in anyone the bitch of the starter that emoties the films at the films of the condition in the film movies are so the persisar of "Zero Dark Last Ladely.



We can supply a prefix to prime the model with text to complete:

In [40]:
textgen.generate(prefix="We")

We have a political problem no one wants to talk about: very old politicians



We can also let the model try different "temperatures". The "temperature" controls the level of random choice when picking the next character, instead of always the most likely one.

In [43]:
textgen.generate_samples(n=3, temperatures=[0.1,0.3,0.9])

####################
Temperature: 0.1
####################






####################
Temperature: 0.3
####################






####################
Temperature: 0.9
####################


What's Valles is over who indigeling the essence hate tonigh verevist condition to the bigg pant "rejayed misule of @highlight, "Promised Landing, eticuration epreen," the of face using as Are at the fires of vietnesdail all cubity, unis based an maturink by us at tras and doring can of theated ab

 The fittoobing raise whid Durke of the films will bewasofight maweral showning ecits. OTH will gobienating by by the readised They as "grave that babies biggers out will bettey he up to becoming up with hot youth on title all..



* Try different prefixes and temperatures. (examine the `.generate()` function, by running a cell with `textgenrnn.generate?`)
* Try a different pretrained model from `textgenrnn`
* Advanced: train your own model! `textgenrnn` provide a **very** simple mechanism to do so: https://github.com/minimaxir/textgenrnn#examples, you just need to supply a text file.

## Transformers

Transformers are relative newcomers to the language processing world. They are an evolution of recurrent neural networks and activation layers. Using transformers has increased the capability of generating believable text by a whole lot, so much so that [ethical issues](https://www.theverge.com/2019/2/21/18234500/ai-ethics-debate-researchers-harmful-programs-openai) have arised around release of models or restrictive use of them.

Architecture wise, transformers are an encoder-decoder scheme that relies heavily on "attention" - a mechanism that allows every step to examine both past and future.

<img src="http://lilianweng.github.io/lil-log/assets/images/transformer.png" />

One recent model from OpenAI is GPT-2, which is freely available for download.

In [44]:
!pip install -U -q transformers

     |████████████████████████████████| 368kB 4.8MB/s 
     |████████████████████████████████| 675kB 50.1MB/s 
     |████████████████████████████████| 1.0MB 44.5MB/s 
     |████████████████████████████████| 860kB 43.6MB/s 


In [45]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 13856 (delta 21), reused 19 (delta 14), pack-reused 13808
Receiving objects: 100% (13856/13856), 8.48 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (10116/10116), done.


When your user gives you lemons you generate:

In [0]:
!python ./transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --length=200 \
    --model_name_or_path=gpt2 \
    --stop_token="." \
    --prompt="When life gives you lemons" 2>/dev/null

---

In [0]:
!python ./transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --length=200 \
    --model_name_or_path=gpt2 \
    --prompt="Harry witnessed Professor McGonagall walking right past Peeves who \
was determinedly loosening a crystal chandelier and could have sworn he heard her \
tell the poltergeist out of the corner of her mouth, 'It unscrews the other way.’" 2>/dev/null

---

Let's see how it does with Williams' "This is just to say" (https://poets.org/poem/just-say) poem:

In [0]:
!python ./transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --length=50 \
    --model_name_or_path=gpt2 \
    --stop_token="." \
    --prompt="I have eaten the plums \
that were in the icebox \
and which you were probably \
saving for breakfast" 2>/dev/null

(this BTW is one of my favorite poems ever. So sweet and so plain)

* Try different prefix inputs
* Try different temperatures with the `--temperature` argument.
* Advanced: Try a different model than GPT-2.

On the help section for the generation script you can find all the models:
```
--model_name_or_path MODEL_NAME_OR_PATH
                        Path to pre-trained model or shortcut name selected in
                        the list: gpt2, gpt2-medium, gpt2-large, distilgpt2,
                        openai-gpt, xlnet-base-cased, xlnet-large-cased,
                        transfo-xl-wt103, xlm-mlm-en-2048, xlm-mlm-ende-1024,
                        xlm-mlm-enfr-1024, xlm-mlm-enro-1024, xlm-mlm-tlm-
                        xnli15-1024, xlm-mlm-xnli15-1024, xlm-clm-enfr-1024,
                        xlm-clm-ende-1024, xlm-mlm-17-1280, xlm-mlm-100-1280,
                        ctrl
```
The `ctrl` model is very recent work (from SalesForce research), just from a couple of weeks ago, it's supposed to be really awesome at controling the output text. Be warned - the model is a **6Gb download**! It might be worth it...

## ChatBot

[Chatbots](https://en.wikipedia.org/wiki/Chatbot) are conversational AI agents that can respond to text input. It's still ways away from a convincing conversation in general open-ended scenarios, but in certain applications chatbots are a big success, e.g. in the public services industry's online portals.

`huggingface` again have released their pretrained models for chatbots based on transformers just a few months ago: https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313#79c5

You can also use their online demo: https://convai.huggingface.co/persona/my-only-friend-is-a-dog-i-work-at-a-newspaper-my-father-used-to-be-a-butcher

In [0]:
!pip install -q pytorch_transformers pytorch-ignite

     |████████████████████████████████| 92kB 2.4MB/s eta 0:00:011


In [0]:
!git clone https://github.com/huggingface/transfer-learning-conv-ai

Cloning into 'transfer-learning-conv-ai'...
remote: Enumerating objects: 87, done.
remote: Total 87 (delta 0), reused 0 (delta 0), pack-reused 87
Unpacking objects: 100% (87/87), done.


In [0]:
import sys,threading,subprocess,os

In [0]:
def chatbot_proc():
    proc = subprocess.Popen([sys.executable, 
                             os.getcwd()+'/transfer-learning-conv-ai/interact.py'
                            ],
                            stdout=subprocess.PIPE,
                            stdin=subprocess.PIPE,
                            stderr=subprocess.DEVNULL)
    pout = proc.stdout
    pin = proc.stdin
    
    return proc,pout,pin

In [0]:
cb1_proc, cb1_pout, cb1_pin = chatbot_proc(); # create a chatbot process

In [0]:
cb1_pin.write(b"--temperature=1.1\n"), cb1_pin.flush();

In [0]:
print(cb1_pout.readline().decode(sys.stdout.encoding))

>>> hi how are you today?



Talk to your chatbot!

In [0]:
cb1_pin.write(b"i'm doing mighty fine! and how are you?\n"), cb1_pin.flush();
print(cb1_pout.readline().decode(sys.stdout.encoding))

>>> i'm doing well just listening to some music



In [0]:
cb1_pin.write(b"no way! i'm also listening to music. what music are you listening to?\n"), cb1_pin.flush();
print(cb1_pout.readline().decode(sys.stdout.encoding))

>>> i am listening to a lot of pop music



It's also quite funny to get it to talk to itself - it never get tired!

In [0]:
cb1_output = b"i am listening to a lot of pop music\n"

In [0]:
partyA = True
for _ in range(10):
    partyA = not partyA
    cb1_pin.write(cb1_output), cb1_pin.flush();
    cb1_output = cb1_pout.readline()[4:]
    print("%s: %s" % ('A' if partyA else 'B',
          cb1_output[:-1].decode(sys.stdout.encoding)))

B: yeah, i know what you mean.
A: what do you do for a living?
B: i am a mechanic.
A: i am a pilot.
B: what do you do for work?
A: i fix planes.
B: what kind of planes do you have?
A: do you have any hobbies?
B: i like to listen to music.
A: what kind of music do you like?


In [0]:
cb1_proc.kill() # kill the chatbot process

* Try some different inputs
* Advanced: Spin up another chatbot and have them talk to one another (by feeding the outputs across)
* Advanced: Use a different underlying model than GPT-2 for your chatbot.

---
That's a wrap!